In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pickle
import pandas as pd

In [4]:
with open('final.pkl', 'rb') as f:
    df = pickle.load(f)

In [5]:
df.head()

,username,eye_color,hair,skin_tone,skin_type,skin_concerns,foundation_color,review,recommend,unhelpful,helpful,star
0,creecreex,Brown,Brunette,Deep,Normal,Cellulite,370,I have 370 and I got matched at Sephora. I lik...,1,0,14,100
1,Jijine,Brown,Black,Dark,Combination,Unknown,470,It is like my skin but better. very lightweigh...,1,0,5,100
2,bloodyblondie,Blue,Black,Porcelain,Dry,Blackheads,100,I was so nervous about trying this product bec...,1,0,5,100
3,miralanani,Brown,Brunette,Medium,Combination,Acne,270,This product was amazing! Just fell in love! T...,1,0,5,100
4,shaaliyah,Brown,Black,Deep,Combination,Acne,370,Since purchasing this foundation it's been my ...,1,0,38,100


# Count Vectorize

In [6]:
from gensim import corpora, models, similarities, matutils
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import logging
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [7]:
stop_words_list=list(ENGLISH_STOP_WORDS.copy())

In [8]:
new_words=['coverage', 'foundation', 'fenty', 'finish', 'does', 've', 'doesn', 'look', 'like', 'product', 'don', 'face', 'just', 'really']
stop_words_list.extend(new_words)

In [9]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),
                                   stop_words=stop_words_list, max_df=0.6, token_pattern="\\b[a-z][a-z]+\\b")

In [10]:
cv_data=count_vectorizer.fit_transform(df['review'])

In [11]:
cv_as_array= count_vectorizer.fit_transform(df['review']).toarray()

In [12]:
dt=pd.DataFrame(cv_as_array, columns=count_vectorizer.get_feature_names())

In [13]:
dt.shape

(2657, 42453)

In [14]:
count_vectorizer.fit(df['review'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.6, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None,
        stop_words=['besides', 'much', 'someone', 'there', 'anyway', 'wherein', 'on', 'as', 'somewhere', 'empty', 'this', 'why', 'might', 'through', 'amongst', 'could', 'con', 'made', 'we', 'via', 'what', 'forty', 'all', 'here', 'couldnt', 'give', 'had', 'sometimes', 'yet', 'myself', 'too', 'herself', 'unde...fenty', 'finish', 'does', 've', 'doesn', 'look', 'like', 'product', 'don', 'face', 'just', 'really'],
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [15]:
counts = count_vectorizer.transform(df['review']).transpose()

In [16]:
counts.shape

(42453, 2657)

In [17]:
corpus = matutils.Sparse2Corpus(counts)

In [18]:
id2word = { identifier: word for word, identifier in count_vectorizer.vocabulary_.items()}

In [19]:
len(id2word)

42453

# LDA

In [20]:
lda = models.LdaModel(corpus=corpus, num_topics=5, minimum_probability=0.03, id2word=id2word, passes=100)

In [21]:
lda.print_topics()

[(0,
  '0.011*"dry" + 0.008*"love" + 0.006*"shade" + 0.006*"oily" + 0.005*"dry skin" + 0.005*"good" + 0.005*"matte" + 0.005*"looks" + 0.004*"use" + 0.004*"perfect"'),
 (1,
  '0.008*"oily" + 0.005*"day" + 0.005*"love" + 0.004*"matte" + 0.004*"work" + 0.004*"did" + 0.004*"dry" + 0.004*"oily skin" + 0.003*"shade" + 0.003*"looked"'),
 (2,
  '0.005*"love" + 0.004*"dry" + 0.004*"tried" + 0.004*"great" + 0.003*"looks" + 0.002*"shade" + 0.002*"use" + 0.002*"makeup" + 0.002*"day" + 0.002*"matte"'),
 (3,
  '0.009*"love" + 0.007*"color" + 0.006*"match" + 0.006*"perfect" + 0.004*"shade" + 0.004*"primer" + 0.004*"great" + 0.004*"amazing" + 0.004*"dry" + 0.003*"day"'),
 (4,
  '0.011*"love" + 0.008*"day" + 0.004*"oily" + 0.004*"long" + 0.004*"matte" + 0.004*"powder" + 0.003*"great" + 0.003*"shade" + 0.003*"primer" + 0.003*"setting"')]

# NMF / SVD

In [22]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [23]:
from sklearn.decomposition import NMF, TruncatedSVD

In [24]:
n_comp = 5
lsa_tfidf = TruncatedSVD(n_components=n_comp)

lsa_tfidf_data = lsa_tfidf.fit_transform(cv_data)

In [25]:
display_topics(lsa_tfidf,count_vectorizer.get_feature_names(),10)


Topic  0
dry, love, oily, shade, matte, day, primer, color, tried, use

Topic  1
dry, dry skin, primer, patches, dry patches, use, good, looked, skin dry, work

Topic  2
love, love love, dry, dry skin, oily, wanted love, oily skin, absolutely love, absolutely, wanted

Topic  3
oily, oily skin, day, matte, hours, end, foundations, nose, end day, skin oily

Topic  4
tried, color, looked, primer, did, work, brush, didn, beauty, wanted
